In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
dataPath = "/root/bc_data/ddsm-png.25/Ddsm_png.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png.25', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [6]:
metaData, meta2 = bc.load_training_metadata(dataPath, balanceViaRemoval=True, verbose=True)

Before Balancing
----------------
benign 3164
malignant 1096

After Balancing
----------------
benign 3164
malignant 1096


In [ ]:
meta2[meta2.keys()[0]]

In [ ]:
# Actually load some representative data for model experimentation
maxData = 1000
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [ ]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [ ]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

## Define and Load Trained Model

In [ ]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

In [ ]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [ ]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [ ]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [ ]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

In [ ]:
model.save_weights("dwdii-bc-150-v0-Cloud.hdf5", overwrite=True)

### Experiment Results

* bc_model_v0 (150x150, 800/200): 182s - loss: 0.0560 - acc: 0.9813 - val_loss: 1.9918 - val_acc: 0.6800